In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load your trained model
model = load_model('improved_brain_tumor_detector.h5')


In [8]:
# Paths to training and testing directories
train_dir = r"C:\Users\tadak\OneDrive\Desktop\Data analysis\work\Challenge\Project 4\Project4\Eric's Work\Sample_BrainTumor_images\Sample_BrainTumor_images\Training"
test_dir = r"C:\Users\tadak\OneDrive\Desktop\Data analysis\work\Challenge\Project 4\Project4\Eric's Work\Sample_BrainTumor_images\Sample_BrainTumor_images\Testing"

# Updated categories with underscores
categories = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']



In [9]:
# Define a function to preprocess images and make predictions
def process_and_predict_images(directory, model, categories):
    results = []  # List to store prediction results
    
    IMG_SIZE = (128, 128)  # Size expected by the model
    
    # Loop through each category
    for category in categories:
        category_path = os.path.join(directory, category)
        
        # Check if the category folder exists
        if not os.path.exists(category_path):
            print(f"Folder not found: {category_path}")
            continue  # Skip to the next category if folder is missing
        
        # Loop through each image in the category folder
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            
            # Load and preprocess the image
            img = cv2.imread(img_path)
            if img is None:
                print(f"Could not load image: {img_path}")
                continue
            
            img = cv2.resize(img, IMG_SIZE)
            img = img_to_array(img) / 255.0
            img = np.expand_dims(img, axis=0)  # Add batch dimension
            
            # Make prediction
            prediction = model.predict(img)
            predicted_label = "Tumor detected" if prediction > 0.5 else "No tumor detected"
            
            # Append results
            results.append({
                "Filename": img_name,
                "True Label": category,
                "Predicted Label": predicted_label,
                "Probability": float(prediction)  # Model's prediction probability
            })
    

In [ ]:
train_results = process_and_predict_images(train_dir, model, categories)
test_results = process_and_predict_images(test_dir, model, categories)

1/1 [==============================] - 0s 18ms/step


C:\Users\tadak\AppData\Local\Temp\ipykernel_53864\229564377.py:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  "Probability": float(prediction)  # Model's prediction probability


1/1 [==============================] - ETA: 0s

In [ ]:
# Convert results to DataFrames
train_results_df = pd.DataFrame(train_results)
test_results_df = pd.DataFrame(test_results)

# Save results to CSV files
train_results_df.to_csv('train_predictions_log.csv', index=False)
test_results_df.to_csv('test_predictions_log.csv', index=False)

print("Training and testing results saved to CSV files.")